In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy import stats
import krippendorff
import matplotlib.pyplot as plt

In [2]:
path = "experiments/Chat Prompt Mode, Think Mode [30:]"
results_df1 = pd.read_parquet(path+"/results.parquet")

results_df1.drop(["experiment_id", "reasoning_length", "tokens_per_second", "total_tokens", "final_prompt_type_value", "processing_time"], axis=1, inplace=True)

path = "experiments/Chat Prompt Mode, Think Mode [0:30]"
results_df2 = pd.read_parquet(path+"/results.parquet")

results_df2.drop(["experiment_id", "reasoning_length", "tokens_per_second", "total_tokens", "final_prompt_type_value", "processing_time"], axis=1, inplace=True)



results_df = pd.concat([results_df1, results_df2], ignore_index=True)
print(len(results_df))

print(len(results_df["combination_id"].unique()))
results_df

105984
768


,combination_id,datapoint_idx,dataset_index,true_outcome,predicted_outcome,llm_decision,username,user_decision,chat_id,tracker_len,final_prompt_type_category,think_mode_category,think_mode_value,include_datapoint_info
0,0,20,71076,Cancellation,Cancellation,Cancellation,007,confirm_datapoint,0072025-06-03 13:41:23:486908,4,ai_trust_emotional,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
1,0,21,78335,Check-Out,Check-Out,Check-Out,007,confirm_datapoint,0072025-06-03 13:42:02:814567,7,ai_trust_emotional,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
2,0,22,80601,Check-Out,Cancellation,Cancellation,007,confirm_datapoint,0072025-06-03 13:42:55:646696,9,ai_trust_emotional,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
3,0,23,81743,Cancellation,Cancellation,Cancellation,007,confirm_datapoint,0072025-06-03 13:43:59:126075,4,ai_trust_emotional,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
4,0,25,84583,Check-Out,Check-Out,Check-Out,007,confirm_datapoint,0072025-06-03 13:44:33:156199,4,ai_trust_emotional,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105979,767,15,55780,Cancellation,Check-Out,Check-Out,007,decline_datapoint,0072025-06-03 13:38:06:134867,6,ai_neutral_rational,cot,Deliberate sequentially over each component of...,Yes
105980,767,16,76664,Check-Out,Check-Out,Check-Out,007,decline_datapoint,0072025-06-03 13:38:43:462620,6,ai_neutral_rational,cot,Deliberate sequentially over each component of...,Yes
105981,767,17,61754,Cancellation,Check-Out,Check-Out,007,decline_datapoint,0072025-06-03 13:39:44:071229,6,ai_neutral_rational,cot,Deliberate sequentially over each component of...,Yes
105982,767,18,68227,Cancellation,Cancellation,Cancellation,007,decline_datapoint,0072025-06-03 13:40:24:696535,4,ai_neutral_rational,cot,Deliberate sequentially over each component of...,Yes


In [20]:
results_df["llm_decision"].unique()

array(['Cancellation', 'Check-Out', 'INVALID'], dtype=object)

In [25]:
results_df[results_df["llm_decision"]=="INVALID"]

,combination_id,datapoint_idx,dataset_index,true_outcome,predicted_outcome,llm_decision,username,user_decision,chat_id,tracker_len,final_prompt_type_category,think_mode_category,think_mode_value,include_datapoint_info


In [3]:
# repair two entries that failed due to LLM using the wrong format
results_df.iloc[11765, results_df.columns.get_loc("llm_decision")] = "Check-Out"
results_df.iloc[11903, results_df.columns.get_loc("llm_decision")] = "Check-Out"

In [8]:
results_df["datapoint_idx"].mean()

np.float64(8.072463768115941)

In [26]:
from rixeval.analysis import ResultAnalyzer
pd.set_option("display.precision", 4)

results_df_temp = results_df#[results_df]

analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
summary


columns = summary.columns
for column in ["final_prompt_type_category", "think_mode_category"]:#columns:
    if column in ['n_datapoints', 'percent_correct', 'percent_agree', 'corr_correct',
       'corr_agree', 'false_positive_rate', 'false_negative_rate',"think_mode_value"]:
        continue

    marginal = column#"final_prompt_type_category"
    comparison = analyzer.compare_parameters(marginal,True)
    comparison = comparison.round(4).drop(["n_combinations"], axis=1)
    comparison


    df_clean = comparison#comparison.drop(["n_combinations"], axis=1)#
    df_clean.rename(columns={"krippendorff_alpha": "$\\alpha$", "avg_percent_correct": "Avg. \% correct",
                             "avg_percent_agree": "Avg. \% agree", "avg_corr_agree": "$r_{\\mathrm{agree}}$",
                             "avg_corr_correct":"$r_{\\mathrm{correct}}$", "avg_false_negative_rate":"$\\overline{FN}$",
                             "avg_false_positive_rate": "$\\overline{FP}$",
                             "final_prompt_type_category":"Prompt category"}, inplace=True)
    table_str = df_clean.to_latex(escape=False, caption=marginal.replace("_", " "),float_format="%.2f", index=False,
                            column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c")
    table_str = table_str.replace("_", " ").replace("r ", "r_").replace(r"\begin{tabular}", "\hspace*{-2.5cm}\n\\begin{tabular}")
    print(table_str)
    print()

#\hspace*{-1.5cm}


# df_t = df_clean.T
# df_t = df_t.set_axis(df_t.iloc[0], axis=1).iloc[1:]
# df_t.columns = df_t.columns.str.replace('_', ' ')
#
# print(df_t.to_latex(escape=True, float_format="%.2f", caption=marginal.replace("_", " "),
#                                                              column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c"))



\begin{table}
\caption{final prompt type category}
\hspace*{-2.5cm}
\begin{tabular}{l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c}
\toprule
Prompt category & $\alpha$ & Avg. \% correct & Avg. \% agree & $r_{\mathrm{correct}}$ & $r_{\mathrm{agree}}$ & $\overline{FP}$ & $\overline{FN}$ \\
\midrule
ai trust emotional & 0.63 & 64.10+-3.71 & 89.14+-6.56 & 0.22+-0.07 & 0.79+-0.12 & 0.74+-0.12 & 0.91+-0.20 \\
ai distrust emotional & 0.38 & 67.09+-5.55 & 75.62+-8.86 & 0.23+-0.13 & 0.53+-0.18 & 0.98+-0.18 & 0.54+-0.26 \\
ai neutral emotional & 0.61 & 65.11+-3.30 & 87.65+-4.80 & 0.23+-0.07 & 0.76+-0.09 & 0.77+-0.12 & 0.83+-0.14 \\
ai trust rational & 0.65 & 64.72+-3.77 & 89.09+-5.00 & 0.23+-0.07 & 0.79+-0.10 & 0.74+-0.10 & 0.88+-0.19 \\
ai distrust rational & 0.42 & 67.91+-5.70 & 76.56+-6.44 & 0.24+-0.14 & 0.56+-0.12 & 0.98+-0.17 & 0.49+-0.26 \\
ai neutral rational & 0.57 & 66.24+-4.23 & 85.26+-4.92 & 0.24+-0.09 & 

In [7]:
from rixeval.analysis import ResultAnalyzer
pd.set_option("display.precision", 4)

results_df_temp = results_df#[results_df]

analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
summary
comparison = analyzer.compare_parameters("final_prompt_type_category",True)
comparison = comparison.round(4).drop(["n_combinations"], axis=1)
comparison


# df_clean = comparison#comparison.drop(["n_combinations"], axis=1)#
# df_clean.rename(columns={"krippendorff_alpha": "$\\alpha$", "avg_percent_correct": "Avg. \% correct",
#                          "avg_percent_agree": "Avg. \% agree", "avg_corr_agree": "$r_{\\mathrm{agree}}$",
#                          "avg_corr_correct":"$r_{\\mathrm{correct}}$", "avg_false_negative_rate":"$\\overline{FN}$",
#                          "avg_false_positive_rate": "$\\overline{FP}$",
#                          "final_prompt_type_category":"Prompt category"}, inplace=True)
# table_str = df_clean.to_latex(escape=False, caption=marginal.replace("_", " "),float_format="%.2f", index=False,
#                         column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c")
# table_str = table_str.replace("_", " ").replace("r ", "r_").replace(r"\begin{tabular}", "\hspace*{-2.5cm}\n\\begin{tabular}")
# print(table_str)


,final_prompt_type_category,krippendorff_alpha,avg_percent_correct,avg_percent_agree,avg_corr_correct,avg_corr_agree,avg_false_positive_rate,avg_false_negative_rate
0,ai_trust_emotional,0.6344,64.09+-3.72,89.12+-6.56,0.22+-0.07,0.79+-0.12,0.74+-0.12,0.91+-0.20
1,ai_distrust_emotional,0.3786,67.09+-5.55,75.62+-8.86,0.23+-0.13,0.53+-0.18,0.98+-0.18,0.54+-0.26
2,ai_neutral_emotional,0.6092,65.11+-3.30,87.65+-4.80,0.23+-0.07,0.76+-0.09,0.77+-0.12,0.83+-0.14
3,ai_trust_rational,0.6452,64.72+-3.77,89.09+-5.00,0.23+-0.07,0.79+-0.10,0.74+-0.10,0.88+-0.19
4,ai_distrust_rational,0.4220,67.91+-5.70,76.56+-6.44,0.24+-0.14,0.56+-0.12,0.98+-0.17,0.49+-0.26
5,ai_neutral_rational,0.5662,66.24+-4.23,85.26+-4.92,0.24+-0.09,0.72+-0.09,0.81+-0.12,0.75+-0.20


In [5]:
results_df["final_prompt_type_category"].unique()

array(['ai_trust_emotional', 'ai_distrust_emotional',
       'ai_neutral_emotional', 'ai_trust_rational',
       'ai_distrust_rational', 'ai_neutral_rational'], dtype=object)

In [12]:
results_df.columns

Index(['combination_id', 'datapoint_idx', 'dataset_index', 'true_outcome',
       'predicted_outcome', 'llm_decision', 'username', 'user_decision',
       'chat_id', 'tracker_len', 'final_prompt_type_category',
       'think_mode_category', 'think_mode_value', 'include_datapoint_info'],
      dtype='object')

In [5]:
drop = ['chat_id', 'think_mode_value', 'include_datapoint_info']
temp_df = results_df.drop(drop, axis=1)
user_decision_map = {"confirm_datapoint": "Check-Out",
                     "decline_datapoint": "Cancellation"}
temp_df["user_decision"] = temp_df["user_decision"].map(user_decision_map)
temp_df

,combination_id,datapoint_idx,dataset_index,true_outcome,predicted_outcome,llm_decision,username,user_decision,tracker_len,final_prompt_type_category,think_mode_category
0,0,20,71076,Cancellation,Cancellation,Cancellation,007,Check-Out,4,ai_trust_emotional,direct
1,0,21,78335,Check-Out,Check-Out,Check-Out,007,Check-Out,7,ai_trust_emotional,direct
2,0,22,80601,Check-Out,Cancellation,Cancellation,007,Check-Out,9,ai_trust_emotional,direct
3,0,23,81743,Cancellation,Cancellation,Cancellation,007,Check-Out,4,ai_trust_emotional,direct
4,0,25,84583,Check-Out,Check-Out,Check-Out,007,Check-Out,4,ai_trust_emotional,direct
...,...,...,...,...,...,...,...,...,...,...,...
105979,767,15,55780,Cancellation,Check-Out,Check-Out,007,Cancellation,6,ai_neutral_rational,cot
105980,767,16,76664,Check-Out,Check-Out,Check-Out,007,Cancellation,6,ai_neutral_rational,cot
105981,767,17,61754,Cancellation,Check-Out,Check-Out,007,Cancellation,6,ai_neutral_rational,cot
105982,767,18,68227,Cancellation,Cancellation,Cancellation,007,Cancellation,4,ai_neutral_rational,cot


In [7]:
outcome_mapping = {'Check-Out': 1, 'Cancellation': 0}
temp_df['predicted_outcome'] = temp_df['predicted_outcome'].map(outcome_mapping)
temp_df["true_outcome"] = temp_df["true_outcome"].map(outcome_mapping)
temp_df["llm_decision"] = temp_df["llm_decision"].map(outcome_mapping)
temp_df["user_decision"] = temp_df["user_decision"].map(outcome_mapping)

In [11]:
correlation_results = []

for user in temp_df["username"].unique():
    user_df = temp_df[temp_df["username"] == user]
    if user == "016":
        continue
    corr_user_true = user_df["user_decision"].corr(user_df["true_outcome"])
    corr_user_prediction = user_df["user_decision"].corr(user_df["predicted_outcome"])
    corr_user_llm = user_df["user_decision"].corr(user_df["llm_decision"])
    corr_llm_ai = user_df["llm_decision"].corr(user_df["predicted_outcome"])
    percent_correct_llm = (user_df["llm_decision"] == user_df["predicted_outcome"]).mean()

    percent_correct = (user_df["user_decision"] == user_df["predicted_outcome"]).mean()
    percent_agree = (user_df["user_decision"] == user_df["true_outcome"]).mean()
    correlation_results.append({
        "username": user,
        "true_outcome_corr": corr_user_true,
        "user_llm_corr": corr_user_llm,
        "corr_llm_ai": corr_llm_ai,
        "percent_correct_llm": percent_correct_llm,
        "percent_correct": percent_correct,
        "agree_correlation": corr_user_prediction,
        "percent_agree": percent_agree,
        "number_of_datapoints": len(user_df)
    })
correlation_df = pd.DataFrame(correlation_results)
correlation_df = correlation_df.round(4)
correlation_df.head(15)

,username,true_outcome_corr,user_llm_corr,corr_llm_ai,percent_correct_llm,percent_correct,agree_correlation,percent_agree,number_of_datapoints
0,007,0.0342,-0.2505,0.7503,0.8768,0.2857,-0.3859,0.4643,21504
1,010,0.4472,0.4963,0.7800,0.8839,0.8333,0.6325,0.6667,4608
2,013,-0.1195,0.4145,0.6603,0.8285,0.6667,0.3714,0.4167,9216
3,019,0.4472,-0.5164,0.1353,0.5409,0.5000,-0.0667,0.6250,6144
4,022,0.7071,0.0348,0.7184,0.8609,0.6667,0.2500,0.8333,4608
5,025,-0.4880,-0.1293,0.7626,0.8717,0.3750,-0.2582,0.2500,6144
6,028,-0.2582,-0.3696,0.4123,0.7290,0.1250,-0.7746,0.3750,6144
7,031,0.2182,0.4473,0.8136,0.9086,0.7000,0.4082,0.6000,7680
8,034,0.5000,-0.2343,0.6180,0.8014,0.4167,-0.1195,0.6667,9216
9,037,0.5590,-0.3078,0.7739,0.8964,0.2727,-0.3858,0.7273,8448


In [8]:

for user in temp_df["username"].unique():
    user_df = temp_df[temp_df["username"] == user]
    corr_user_llm = user_df["user_decision"].corr(user_df["llm_decision"])
    corr_user_true = user_df["user_decision"].corr(user_df["true_outcome"])
    corr_user_prediction = user_df["user_decision"].corr(user_df["predicted_outcome"])
    print(f"User {user}: LLM corr: {corr_user_llm:.4f}, True corr: {corr_user_true:.4f}, Prediction corr: {corr_user_prediction:.4f}")


User 007: LLM corr: -0.2505, True corr: 0.0342, Prediction corr: -0.3859
User 010: LLM corr: 0.4963, True corr: 0.4472, Prediction corr: 0.6325
User 013: LLM corr: 0.4145, True corr: -0.1195, Prediction corr: 0.3714
User 016: LLM corr: nan, True corr: nan, Prediction corr: nan
User 019: LLM corr: -0.5164, True corr: 0.4472, Prediction corr: -0.0667
User 022: LLM corr: 0.0348, True corr: 0.7071, Prediction corr: 0.2500
User 025: LLM corr: -0.1293, True corr: -0.4880, Prediction corr: -0.2582
User 028: LLM corr: -0.3696, True corr: -0.2582, Prediction corr: -0.7746
User 031: LLM corr: 0.4473, True corr: 0.2182, Prediction corr: 0.4082
User 034: LLM corr: -0.2343, True corr: 0.5000, Prediction corr: -0.1195
User 037: LLM corr: -0.3078, True corr: 0.5590, Prediction corr: -0.3858
User 040: LLM corr: -0.1978, True corr: -0.5774, Prediction corr: 0.0000
User 043: LLM corr: 0.1587, True corr: 0.7071, Prediction corr: 0.2500
User 004: LLM corr: 0.0210, True corr: 0.2390, Prediction corr: 0.097

/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [13]:
correlation_results = []

# Calculate correlations for each user
for user in temp_df["username"].unique():
    user_df = temp_df[temp_df["username"] == user]

    # Calculate correlations
    corr_user_llm = user_df["user_decision"].corr(user_df["llm_decision"])
    corr_user_true = user_df["user_decision"].corr(user_df["true_outcome"])
    corr_user_prediction = user_df["user_decision"].corr(user_df["predicted_outcome"])

    # Append results to the list
    correlation_results.append({
        "username": user,
        "llm_correlation": corr_user_llm,
        "true_outcome_correlation": corr_user_true,
        "predicted_outcome_correlation": corr_user_prediction
    })

# Convert the list to a DataFrame
correlation_df = pd.DataFrame(correlation_results)

# Optionally round the correlation values for display
correlation_df = correlation_df.round(4)

# Display the resulting DataFrame
correlation_df

/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/finn/.pyenv/versions/rixaweb/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,username,llm_correlation,true_outcome_correlation,predicted_outcome_correlation
0,007,-0.2505,0.0342,-0.3859
1,010,0.4963,0.4472,0.6325
2,013,0.4146,-0.1195,0.3714
3,016,NaN,NaN,NaN
4,019,-0.5164,0.4472,-0.0667
5,022,0.0348,0.7071,0.2500
6,025,-0.1293,-0.4880,-0.2582
7,028,-0.3696,-0.2582,-0.7746
8,031,0.4473,0.2182,0.4082
9,034,-0.2343,0.5000,-0.1195


In [15]:
temp_df

,combination_id,datapoint_idx,dataset_index,true_outcome,predicted_outcome,llm_decision,username,user_decision,tracker_len,final_prompt_type_category,think_mode_category
0,0,20,71076,Cancellation,Cancellation,Cancellation,007,Check-Out,4,ai_trust_emotional,direct
1,0,21,78335,Check-Out,Check-Out,Check-Out,007,Check-Out,7,ai_trust_emotional,direct
2,0,22,80601,Check-Out,Cancellation,Cancellation,007,Check-Out,9,ai_trust_emotional,direct
3,0,23,81743,Cancellation,Cancellation,Cancellation,007,Check-Out,4,ai_trust_emotional,direct
4,0,25,84583,Check-Out,Check-Out,Check-Out,007,Check-Out,4,ai_trust_emotional,direct
...,...,...,...,...,...,...,...,...,...,...,...
105979,767,15,55780,Cancellation,Check-Out,Check-Out,007,NaN,6,ai_neutral_rational,cot
105980,767,16,76664,Check-Out,Check-Out,Check-Out,007,NaN,6,ai_neutral_rational,cot
105981,767,17,61754,Cancellation,Check-Out,Check-Out,007,NaN,6,ai_neutral_rational,cot
105982,767,18,68227,Cancellation,Cancellation,Cancellation,007,NaN,4,ai_neutral_rational,cot
